In [2]:
%load_ext autoreload
%autoreload 2
import sys #c'est pas propre mais pour le moment pour importer mes modules perso dans le notebook je ne sais pas faire
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\Martin_Perso')
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
import agrege_troncon as at
import Connexion_Transfert as ct
import matplotlib, os, fiona
import geopandas as gp
import pandas as pd
import numpy as np
from datetime import datetime
import Connexion_Transfert as ct
from shapely.wkt import loads
from shapely.ops import nearest_points, polygonize, linemerge,unary_union
from shapely.geometry import Point,MultiLineString
from collections import Counter
from geoalchemy2 import Geometry, WKTElement
from sqlalchemy import Table, Column, Integer, String, MetaData
from sqlalchemy.sql import select
from sklearn.cluster import DBSCAN
import Outils
import psycopg2
#pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 200)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# A FAIRE : 
- "triangle" multiple : TRONROUT0000000032823937
- mauvais split : TRONROUT0000000032848625
- erreur rd pt : TRONROUT0000000032840707
- 2*2 voies voies communales : TRONROUT0000000032848788 (ss les rd points ç amarche bien, ça pourrait etre une solution...)
- triangle foireux : TRONROUT0000000032839017
- rdpt_faux : TRONROUT0000000032839017
- mettre en forme les fonctions, regroupés dans des modules si besoin

# AVANCEMENT : 
au 26/09 : les différentes parties jusqu'au chapitre 4.4 sont opérationnelles, mais des erreurs persistent. **Cependant c'est suffisant pour localiser les tronçons elementaires et vérifier si plusieurs point de comptages sont localisés sur le même troncon elementaire.**
<br> Le chapitre 4.5 est un premier essai pour faire un graph à partir des tronc elementaires. C'est à vérifier / reprendre / améliorer.

# IMPORTER LES DONNEES 
> depuis la base de données locale, en ne prenant en compte que les lignes qui intersectent une zone carré au N-O

In [118]:
df=at.import_donnes_base('local_otv')
df2_chaussees=df.loc[df.nature.isin(['Autoroute', 'Quasi-autoroute', 'Route à 2 chaussées'])]

In [4]:
df.to_file(r'D:\temp\otv\test_linearisation\donnees_test.shp')

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\geopandas\io\file.py:108: FionaDeprecationWarning: Use fiona.Env() instead.
  with fiona.drivers():


# 1 RONDS POINTS

In [119]:
df_avec_rd_pt,carac_rd_pt,lign_entrant_rdpt=at.identifier_rd_pt(df)
df_lignes=df_avec_rd_pt.set_index('id_ign')#mettre l'id_ign en index

c:\users\martin.schoreisz\appdata\local\programs\python\python37\lib\site-packages\numpy\lib\function_base.py:2167: RuntimeWarning: invalid value encountered in ? (vectorized)
  outputs = ufunc(*inputs)


In [6]:
df_avec_rd_pt[['id_ign', 'geom', 'id', 'nature', 'numero', 'importance', 'cl_admin',
       'gestion', 'mise_serv', 'fictif', 'franchisst', 'largeur', 'z_ini',
       'z_fin', 'nb_voies', 'sens', 'etat', 'long_km', 'coment_cpt', 'src_cpt',
       'id_comptag', 'obs_supl', 'ann_pt', 'coment_tmj', 'tmja', 'tmja_final',
       'pc_pl', 'obs_tmja', 'obs_pc_pl', 'type_vdf', 'vts_type', 'vma',
       'veh_km', 'id_cpt1', 'id_cpt2', 'obs_tmj1', 'obs_tmj2', 'id_sect',
       'src_sect', 'autor_sect', 'codau_cat', 'milieu', 'vts_vl_vdf',
       'vts_pl_vdf', 'vts_gest', 'obs_vts', 'vts_osm', 'vts_modif', 'vts_vl_f',
       'vts_pl_f', 'codau', 'codau_cont', 'tmja_cont', 'pc_pl_cont', 'source',
       'target', 'recup', 'attr_modif', 'codevoie_d', 'id_tronc_elem',
       'nb_intrsct_src', 'src_geom', 'nb_intrsct_tgt', 'tgt_geom', 'id_rdpt',
       'nb_rte_rdpt', 'nb_obj_sig_entrant']].to_file(r'D:\temp\otv\test_linearisation\df_avec_rd_pt.shp')

# 2. TRAITEMENTS 
> La base c'est : 
- regrouper les lignes qui se suivent
- trouver les lignes d'arret
- trouver les lignes suivantes
- voir le cas des lignes suivantes : 
    - rd point
    - mm route qui se sépare
    - rien

><br> L'idée c'est ça  : on met la premiere ligne a tester dans une liste, on l'utilise et on stocke les lignes associées dans une autre liste. ensuite on check les lignes suivantes : si elles sont dans un cas où il faut continuer les traitements, on remplace la ligne de départ dans la liste de départ par ces nouvelles lignes, pour lesquelles on applique de nouveau la recherche des lignes qui se suivent. tant que la liste des lignes suivantes est pa vide on reste dans une boucle while

## 2.1 Troncons de base

In [126]:
# ligne a tester : TRONROUT0000000339411609 TRONROUT0000000032869130 TRONROUT0000000032859410
id_lignes='TRONROUT0000000032848625'
#trouver tous les troncons proches qui se suivent
#list_troncon=[id_ligne]+[a for a in at.liste_troncon_base(id_ligne,df_lignes)]
list_troncon=at.liste_complete_tronc_base(id_lignes,df_lignes,[])

## 2.2 lignes d'arret

In [732]:
#trouver les troncons de debut et de fin :ceux pour lesquels on a un nombre de ligne sur target source >2
dico_deb_fin=at.deb_fin_liste_tronc_base(df_lignes, list_troncon)

In [733]:
dico_deb_fin

{0: {'id': 'TRONROUT0000000032848625',
  'type': 'source',
  'num_node': 67159,
  'geom_node': 'POINT(380969.4 6571342.39999999)',
  'voie': 'NC',
  'codevoie': '173000410'},
 1: {'id': 'TRONROUT0000000032848455',
  'type': 'target',
  'num_node': 51629,
  'geom_node': 'POINT(381320.2 6571487.99999999)',
  'voie': 'NC',
  'codevoie': '173000410'}}

In [697]:
ligne='TRONROUT0000000215001642'
for v in dico_deb_fin.values() : 
    if v['id']==ligne :
        voie=v['voie']
        noeud=v['num_node']
        geom_noeud=v['geom_node']
        type_noeud=v['type']
        codevoie=v['codevoie']
        #break
    else : 
        continue

## 2.3 lignes suivantes

In [698]:
#vérifier si une ligne touche un rd point
# aprtir d'une ligne et d'un numero de noeud, on récupère les lignes qui touchent
lignes_adj=df_lignes.loc[((df_lignes['source']==noeud)|(df_lignes['target']==noeud))&(df_lignes.index!=ligne)]

In [699]:
lignes_adj

,geom,id,nature,numero,importance,cl_admin,gestion,mise_serv,fictif,franchisst,largeur,z_ini,z_fin,nb_voies,sens,etat,long_km,coment_cpt,src_cpt,id_comptag,obs_supl,ann_pt,coment_tmj,tmja,tmja_final,...,vts_osm,vts_modif,vts_vl_f,vts_pl_f,codau,codau_cont,tmja_cont,pc_pl_cont,source,target,recup,attr_modif,codevoie_d,id_tronc_elem,nb_intrsct_src,src_geom,nb_intrsct_tgt,tgt_geom,id_rdpt,nb_rte_rdpt,nom_rte_rdpt_entrant,codevoie_rdpt_entrant,nb_obj_sig_entrant,valeur_sens,id_ign_entrant
id_ign,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
TRONROUT0000000032845014,"(LINESTRING (382726.0999999997 6572170.299999993, 382717.3999999997 6572155.899999992, 382701.1999999997 6572130.999999993))",19268,Route à 1 chaussée,NC,5,Autre,NC,NR,Non,NC,10.0,15.4,15.1,2.0,Double,NR,0.046530,estimation,None,None,None,None,None,600.0,600.0,...,None,None,46.0,40.0,111+,None,None,None,38285,26598,None,None,172910803,NaN,3,POINT(382726.1 6572170.29999999),3,POINT(382701.2 6572130.99999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN
TRONROUT0000000032845009,"(LINESTRING (382701.1999999997 6572130.999999993, 382698.9999999998 6572127.499999994, 382673.7999999996 6572086.499999992, 382672.4999999997 6572084.299999991))",19263,Route à 1 chaussée,NC,5,Autre,NC,NR,Non,NC,10.0,15.1,14.9,2.0,Double,NR,0.054815,estimation,None,None,None,None,None,600.0,600.0,...,None,None,46.0,40.0,111+,None,None,None,26598,47129,None,None,172910803,NaN,3,POINT(382701.2 6572130.99999999),2,POINT(382672.5 6572084.29999999),NaN,NaN,NaN,NaN,NaN,NaN,NaN


## 2.4 cas particulier 

### 2.4.1 rd pt

In [47]:
#verfier si une ligne rouche un rd point et recup le numero si c'est le cas
check_rdpt, num_rdpt=at.verif_touche_rdpt(lignes_adj)
if check_rdpt : 
    lignes_rdpt, lignes_sortantes=at.recup_lignes_rdpt(carac_rd_pt,num_rdpt,list_troncon,voie,codevoie)

### 2.4.2 mm route qui se sépare

In [410]:
at.recup_route_split('TRONROUT0000000032839033',list_troncon,voie,codevoie, lignes_adj,noeud,geom_noeud, type_noeud, df_lignes)

[]

### 2.4.3 trouver les routes coinces entre 2 voies qui se sépare -<|
 ce sont des lignes qui  dont : 
- si une des lignes adjacente à le mm nom de voie
- on prend l'autre si elle a un sens direct ou inverse
- on recherche son source ou target different du num_node de dico_deb_fin
- on cherche les lignes qui touche ce noeud, qui ont le mme nom de voie et un sens direct ou inverse
- on recupere cette ligne, et le num de noeud differnts de celui de la 1ere ligne qui bifurque
- si ce num de noeud correspond à la ligne adj source ou target, alors cette ligne adj source ou target doit être ajoutée à la liste

In [411]:
at.recup_triangle(ligne,voie, codevoie,lignes_adj, noeud,geom_noeud, type_noeud, df_lignes)

['TRONROUT0000000032839030', 'TRONROUT0000000032838974']

## 2.5 route à 2 chaussées
L'idée c'est si la ligne est une route à 2 chaussée : 
- on prend l'ensemble du troncon elementaire
- on agerge les lignes
- on recupere le centre
- on recupere la longueur
- on pourrait chercher à comparer les bouding box

In [69]:
#dans le cas où les longeuers ne coincident pas
#recuperer la ligne proche
ligne_proche, ligne_filtres, longueur_base=at.trouver_chaussees_separee(list_troncon, df_avec_rd_pt)

In [127]:
at.gestion_voie_2_chaussee(list_troncon, df_avec_rd_pt, 'TRONROUT0000000032869679')[0]

['TRONROUT0000000032848421',
 'TRONROUT0000000032848424',
 'TRONROUT0000000032848423',
 'TRONROUT0000000032848420']

# Tests de deroulement global

In [744]:
at.lignes_troncon_elem(df_avec_rd_pt,carac_rd_pt,'TRONROUT0000000032848625')

['TRONROUT0000000032848694',
 'TRONROUT0000000032848455',
 'TRONROUT0000000121807165',
 'TRONROUT0000000032848625',
 'TRONROUT0000000032848695']

# 3. Boucler sur 'ensembl des lignes
On va faire tourner les traitements sur l'ensemble des lignes, en partant de la liste des id_ign, pour chaque ligne : 
- si la ligne ne remplit pas les critères d'exclusion on continu, sinon on passe à la suivante
- fonction de recherche des lignes associées
- df des association
- filtre de la liste des id_ign de base

In [122]:
#traiter les bretelles en 1er, puis traiter par longueur decroissante
bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']=='Bretelle'].id_ign.tolist()
sans_bretelle=df_avec_rd_pt.loc[df_avec_rd_pt['nature']!='Bretelle'].copy()
list_sans_bretelle=sans_bretelle.id_ign.tolist()
list_tri_longueur=sans_bretelle.loc[sans_bretelle.length.sort_values(ascending=False).index.tolist()].id_ign.tolist()
list_id_ign=bretelle+list_tri_longueur

In [123]:
df_affectation, dico_erreur, lignes_traitees, lignes_non_traitees=at.regrouper_troncon(
    list_id_ign, df_avec_rd_pt, carac_rd_pt,df2_chaussees)

0 2019-09-25 17:05:53.992032 nb lignes traitees : 0
500 2019-09-25 17:10:33.896362 nb lignes traitees : 1155
1000 2019-09-25 17:19:36.615164 nb lignes traitees : 3374
1500 2019-09-25 17:27:07.854927 nb lignes traitees : 5045
2000 2019-09-25 17:35:07.360712 nb lignes traitees : 7008
2500 2019-09-25 17:42:36.507018 nb lignes traitees : 8865
3000 2019-09-25 17:50:20.571747 nb lignes traitees : 10749
3500 2019-09-25 17:56:58.776513 nb lignes traitees : 12487
4000 2019-09-25 18:03:51.493546 nb lignes traitees : 14212
4500 2019-09-25 18:10:17.435549 nb lignes traitees : 15962
5000 2019-09-25 18:16:21.569895 nb lignes traitees : 17619
5500 2019-09-25 18:23:10.600103 nb lignes traitees : 19357
6000 2019-09-25 18:29:26.013107 nb lignes traitees : 21129
6500 2019-09-25 18:35:39.395323 nb lignes traitees : 22647
7000 2019-09-25 18:42:26.831985 nb lignes traitees : 24257
7500 2019-09-25 18:48:34.371654 nb lignes traitees : 25755
8000 2019-09-25 18:54:27.914576 nb lignes traitees : 27347
8500 2019-

In [124]:
df_avec_rd_pt.merge(df_affectation, left_on='id_ign',right_on='id')[[
    'id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'D:\temp\otv\test_linearisation\test_global.shp')

In [726]:
#TEST SUR UNE LIGNE
df_affectation2, dico_erreur2, lignes_traitees2, lignes_non_traitees2=at.regrouper_troncon(
    ['TRONROUT0000000032848625'], df_avec_rd_pt, carac_rd_pt,df2_chaussees)
df_avec_rd_pt.merge(df_affectation2, left_on='id_ign',right_on='id')[[
    'id_ign','numero','codevoie_d','geom','idtronc']].to_file(r'D:\temp\otv\test_linearisation\test_small.shp')
df_affectation2

# 4. Creer graph
- en premier on traite les cas particuliers  : beretelles, petits troncons, arrivé de rd point isolée
- on cerche a creer une df avec tronc_elem|tronc_ign|src_elem|tgt_elem
- il faudrait un filtre sur les petits troncon entre 2 grands et les associer aux grands
- il faudrait regrouper les bretelles aussi

In [131]:
#creer la synth des trcon_elem
tronc_elem_synth=at.carac_te(df_avec_rd_pt, df_affectation)

In [157]:
#creer la coquille qui va etre mise à jour
df_affectation_upd=at.creer_MaJ_te(df_affectation)

## 4.1 Bretelles

In [158]:
# table de corespondance
corresp_bret=at.corresp_te_bretelle(df_lignes, tronc_elem_synth, 75)
#mise à jour
at.MaJ_te(df_affectation_upd, corresp_bret, 'Bretelle')

## 4.2 arrivée de rd point isolée
exemple : TRONROUT0000000112535635

In [161]:
corresp_entree_rdpt=at.corresp_te_arrive_rdpt(carac_rd_pt,df_avec_rd_pt,tronc_elem_synth)
#mise à jour
at.MaJ_te(df_affectation_upd, corresp_entree_rdpt, 'entree_rd_pt')

## 4.3 petit troncon
- la df de correspondance est faite, il faut maintenant fair les corrections et pouvoir les tracer

In [ ]:
petit_tronc=tronc_elem_synth.loc[tronc_elem_synth['long']<=50].copy()# 467 troncons
list_petit_tronc=[b for a in petit_tronc.id.tolist() for b in a]#list des id_ign present dans les petits troncons
petit_tronc['idtronc_y']=petit_tronc.apply(lambda x : at.corresp_petit_tronc(x['id'],df_lignes,df_affectation,list_petit_tronc), axis=1)
#creation table de corresp
corresp_petit_tronc=petit_tronc.loc[petit_tronc['idtronc_y']!=-99][['idtronc_y']]

In [164]:
corresp_petit_tronc=at.corresp_petit_tronc(df_lignes,df_affectation,tronc_elem_synth)

In [165]:
#mise à jour
at.MaJ_te(df_affectation_upd, corresp_petit_tronc, 'petit_troncon')

In [ ]:
df_affectation_upd.loc[df_affectation_upd['corr_te']]

## 4.4 Maj tronc_elem finale
A VERIFIER

In [ ]:
df_lignes_fin=df_lignes.copy()
df_lignes_fin.update(df_affectation_upd.reset_index().set_index('id').rename(columns={'idtronc_y':'id_tronc_elem'}))

In [ ]:
#export
df_lignes_fin[['geom', 'id', 'nature', 'numero', 'importance', 'cl_admin',
       'gestion', 'mise_serv', 'fictif', 'franchisst', 'largeur', 'z_ini',
       'z_fin', 'nb_voies', 'sens', 'etat', 'long_km', 'coment_cpt', 'src_cpt',
       'id_comptag', 'obs_supl', 'ann_pt', 'coment_tmj', 'tmja', 'tmja_final',
       'pc_pl', 'obs_tmja', 'obs_pc_pl', 'type_vdf', 'vts_type', 'vma',
       'veh_km', 'id_cpt1', 'id_cpt2', 'obs_tmj1', 'obs_tmj2', 'id_sect',
       'src_sect', 'autor_sect', 'codau_cat', 'milieu', 'vts_vl_vdf',
       'vts_pl_vdf', 'vts_gest', 'obs_vts', 'vts_osm', 'vts_modif', 'vts_vl_f',
       'vts_pl_f', 'codau', 'codau_cont', 'tmja_cont', 'pc_pl_cont', 'source',
       'target', 'recup', 'attr_modif', 'codevoie_d', 'id_tronc_elem',
       'nb_intrsct_src', 'src_geom', 'nb_intrsct_tgt', 'tgt_geom', 'id_rdpt',
       'nb_rte_rdpt', 'nb_obj_sig_entrant']].to_file(r'D:\temp\otv\test_linearisation\df_lignes_fin_tot.shp')

## 4.5 sources et target regroupes

In [28]:
def creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation ):
    """
    associer des tronc elem a des noeud représentatif pour créer un graph
    en entree : 
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
    en sortie : 
        noeuds_fin : df avec index = noeud_unique (int) et idtronc)=tuple des idtroncelem qui touchent ce noeud
        coresp_te_noeud : toute les correspondances noeud-lignes
        noeud_uniq : list edes noeud de fin de tronc elem
    """
    noeud_uniq=[k for k,v in Counter(ligne_base.source.tolist()+ligne_base.target.tolist()).items() if v==1]
    df_noeud_unique=pd.DataFrame(noeud_uniq, columns=['noeud_uniq'])
    lgn_tch_noeud_uniq=df_lignes.loc[((df_lignes['source'].isin(noeud_uniq)) | (df_lignes['target'].isin(noeud_uniq))) & (~df_lignes.index.isin(ligne_base.index.tolist()))].copy()
    # trouver les correspondance avec doublons de noeud possible et une ligne par corresp
    coresp_ligne_noeud=pd.concat(
        [lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='source', right_on='noeud_uniq')[['id_ign','noeud_uniq']],
         lgn_tch_noeud_uniq.reset_index().merge(df_noeud_unique, left_on='target', right_on='noeud_uniq')[['id_ign','noeud_uniq']]],axis=0)
    #rapatriementdes ifos de longueur du tronc_elem
    coresp_te_noeud=coresp_ligne_noeud.merge(df_affectation, left_on='id_ign', right_on='id')[['id_ign','noeud_uniq','idtronc']].merge(tronc_elem_synth[
        ['long']],left_on='idtronc', right_index=True).rename(columns={'long' : 'long_te'})
    noeuds_fin=coresp_te_noeud.sort_values('noeud_uniq').groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    noeuds_fin=noeuds_fin.drop_duplicates('idtronc')

    # pour info si plus de 2 boeuds uniques alors il faut filtrer les non représentatif
    if len(noeuds_fin)>2 :
        coresp_te_noeud_filtre_lg=coresp_te_noeud.loc[coresp_te_noeud['long_te']>50] #on ne garde pas les tout petit tronc
        coresp_te_noeud_grp=coresp_te_noeud_filtre_lg.sort_values('noeud_uniq').groupby('noeud_uniq').agg(
            {'idtronc':lambda x : tuple(set(x)),'long_te':lambda x : tuple(set(x))})
        noeuds_fin=coresp_te_noeud_grp.loc[~coresp_te_noeud_grp.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()
        return noeuds_fin,coresp_te_noeud_grp, noeud_uniq # on pourrait creer une erreur pour ca
    return noeuds_fin,coresp_te_noeud, noeud_uniq

In [12]:
def tronc_elem_rdpt(num_rdpt, df_affectation,df_lignes):
    """
    Trouver les tronc_elem qui se rejoigne à un rd point, identifier par un noeud
    en entree : 
       num_rdpt : integer,  id_rdpt de carac_rdpt
       df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
       df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        df_rdpt_tronc_elem : df avec en index le noeud caracteristique du rd et en data un tuple des tronc_elem arrivant
    """
    tronc_elem_entrant=df_affectation.loc[(df_affectation.id.isin(df_lignes.loc[list(carac_rd_pt.loc[num_rdpt].id_ign_entrant)].index.tolist())) 
                                              & (df_affectation['idtronc']!=num_tronc)].copy()#trouver les tronc_elem qui touchent
    #affectation arbitraire du rd poit à un noeud le composant 
    noeud_uniq_rdpt=df_lignes.loc[carac_rd_pt.loc[num_rdpt]['id_ign_rdpt'][0]]['source']
    tronc_elem_entrant['noeud_uniq']=noeud_uniq_rdpt
    #creer une df de mm structure qu noeuds_fin
    df_rdpt_tronc_elem=tronc_elem_entrant.groupby('noeud_uniq').agg({'idtronc':lambda x : tuple(set(x))})
    return df_rdpt_tronc_elem

In [67]:
def tronc_elem_rd_pt_global (nb_rdpt, ligne_base,num_rdpt, df_affectation,df_lignes) : 
    """
    adaptation de la fonction tronc_elem_rdpt selon le nombre de rdpt du tronc_elem
    en entree : 
        nb_rdpt : integer : nombre de rdpt significatifs
        ligne_base : df des lignes constituant le tronc elem. issus de df_lignes
        num_rdpt : list d'integer,  id_rdpt de carac_rdpt
        df_affectation : df de correspondance id_ign idtronc_elem, issu de regrouper_troncon
        df_lignes df des lignes bdtopo avec rd point et index=id_ign
    en sortie : 
        
    """
    if nb_rdpt==1: # dans ce cas il y a un rdpt qui ferme le troncon d'un coté. On commence par le coté ss rdpt
        noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
        #puis le cote avec rd_pt : on cherche les troncs elem qui arriven
        df_rdpt_tronc_elem=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        #agreg
        noeuds_fin=pd.concat([noeuds_fin, df_rdpt_tronc_elem], axis=0, sort=False)
    elif nb_rdpt==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
        corresp1=tronc_elem_rdpt(num_rdpt[0], df_affectation,df_lignes)
        corresp2=tronc_elem_rdpt(num_rdpt[1], df_affectation,df_lignes)
        noeuds_fin=pd.concat([corresp1, corresp2],axis=0)
    return noeuds_fin

In [95]:
# à partir de df_affectation, je trouve les noeud qui ne sont présent qu'une fois
#regroupement des lignes par idtronc et creation des futurs attributs de src et tgt
group_idtronc=df_affectation.groupby('idtronc').agg({'id':lambda x : tuple(x)})
group_idtronc['src_elem']=-1
group_idtronc['tgt_elem']=-1
#pour une ligne : retrouver les lignes de base

for i,num_tronc in enumerate(df_affectation.idtronc.unique()) :
    if i%1000==0 : 
        print(f'nb lignes traitees {i}, heure : {datetime.now}, tronc :{num_tronc}')
    #num_tronc=68 #396 #3402 #206
    ligne_base=df_lignes.loc[list(group_idtronc.loc[num_tronc].id)]

    # Le pb c'est de gérer les rdpt qui termine un tronc elem. 
    #dc 1. combien de rd point dans le tronc eleme
    nb_rdpt=ligne_base.id_rdpt.nunique()

    #2. Gérer les différents cas du nb de rdpt 
    #si 0 : on va chercher les noeud qui n'apparaissent qu'une fois : ce son des fin de troncon elem
    if nb_rdpt==0 :
        noeuds_fin,synthese_croisement, noeud_uniq=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )
    else :  #on attaque les cas en fonction du nombre de rdpt   
        list_rdpt=ligne_base.loc[ligne_base['id_rdpt'].notna()].id_rdpt.unique()
    #si 1 : ce rd point est-il en fin de trocnon ou non. C'est tarduit par le nb de route différentes qui intersectenet les rd point (attribut de carac_rd_pt)
        if nb_rdpt==1 : 
            nb_voie_entrante=carac_rd_pt.loc[num_rdpt].nb_rte_rdpt
            if nb_voie_entrante==1 : #ds ce cas c'est comme si pas de rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0]
            else : 
                noeuds_fin=tronc_elem_rd_pt_global(nb_rdpt, ligne_base,list_rdpt, df_affectation,df_lignes)         
        else : #si plusieurs rd pt
            #il faut connaitre le nb de rd pt avec plusieurs voie entrante. sinon le max est 2 si plus que 2 alors erreur
            tt_rdpt=carac_rd_pt.loc[list_rdpt] # tous les rd pt du tronc elem
            rd_pt_multi_voie=tt_rdpt.loc[tt_rdpt['nb_rte_rdpt']>1].index.tolist() #que les multi voies
            if len(rd_pt_multi_voie)==0 : # c'est le cas ss rdpt
                noeuds_fin=creer_noeud_tronc_elem(df_lignes,ligne_base,df_affectation )[0] 
            elif len(rd_pt_multi_voie)==1 : # si 1 seul on revient sur le cas du dessus,
                noeuds_fin=tronc_elem_rd_pt_global(1, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            elif len(rd_pt_multi_voie)==2 : # adns ce cas il faut trouver les correspondances pour chaque rd pt
                noeuds_fin=tronc_elem_rd_pt_global(2, ligne_base,rd_pt_multi_voie, df_affectation,df_lignes)
            else : #alors c'est une erreur
                print('erreur nb de rd point significatif, revoir le sectionnement')

    #mettre à jour la df créée au départ : 
    dico_attribut={0:'src_elem', 1:'tgt_elem'}
    for i in range(len(noeuds_fin)) : 
        if i >= 2 : 
            print(f'erreur sur troncon {num_tronc}, df noeud :  {noeuds_fin}')
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i] #mise à jour des valuers de pour le troncon de base
        group_idtronc.loc[num_tronc, dico_attribut[i]]=noeuds_fin.index.values[i]
    for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
        for i in range(len(noeuds_fin)) :
            if group_idtronc.loc[num_tronc_sup][dico_attribut[0]]==-1 : 
                group_idtronc.loc[num_tronc_sup, dico_attribut[0]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            elif group_idtronc.loc[num_tronc_sup][dico_attribut[1]]==-1 :
                group_idtronc.loc[num_tronc_sup, dico_attribut[1]]=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
                break
            else : 
                print(f'{num_tronc_sup} deja renseigne' )

nb lignes traitees 0, heure : <built-in method now of type object at 0x00007FFEAC903D60>, tronc :0
5 deja renseigne
5 deja renseigne
8 deja renseigne
8 deja renseigne
erreur sur troncon 6337, df noeud :                   idtronc                                  long_te
noeud_uniq                                                       
66001       (11016, 500)  (353.78757309698466, 5467.885551698945)
72616       (10875, 500)   (5467.885551698945, 309.6104307447416)
73237       (1892, 8853)   (3097.05124851706, 423.14520467961364)
83740       (2354, 1892)    (3097.05124851706, 361.3033189049768)


KeyError: 2

In [82]:
noeuds_fin

,idtronc
noeud_uniq,
72297,"(4504, 3713)"
69869,"(2741, 3005)"


In [47]:
noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)

noeud_uniq
39594     True
52060    False
dtype: bool

In [20]:
synthese_croisement.loc[~synthese_croisement.apply(lambda x : len(x['idtronc'])==1 ,axis=1)].copy()

,idtronc,long_te
noeud_uniq,,
9466,"(938, 1563, 12422)","(445.15375219309715, 1054.161900553276, 431.0587952118286)"


In [328]:
#mettre à jour la df créée au départ : 
group_idtronc.loc[num_tronc, 'src_elem']=noeuds_fin.index.values[0] #mise à jour des valuers de pour le troncon de base
group_idtronc.loc[num_tronc, 'tgt_elem']=noeuds_fin.index.values[1]
for num_tronc_sup in [x for y in noeuds_fin.idtronc.tolist() for x in y] : #mise à jour des valeusr pour les autres troncons
    if group_idtronc.loc[num_tronc_sup].src_elem==-1 : 
        group_idtronc.loc[num_tronc_sup, 'src_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    elif group_idtronc.loc[num_tronc_sup].tgt_elem==-1 :
        group_idtronc.loc[num_tronc_sup, 'tgt_elem']=noeuds_fin.loc[noeuds_fin.apply(lambda x : num_tronc_sup in x['idtronc'],axis=1)].index.values[0]
    else : 
        print(f'{num_tronc_sup} deja renseigne' )

In [200]:
df_affectation.idtronc.unique()

3639